In [6]:
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt

e:\conda\env\lamanv\project1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Carregando os dados
data = pd.read_csv('caminho/do/seu/arquivo.csv')

# Renomeando as colunas conforme o esperado pelo Prophet
data = data.rename(columns={'Date': 'ds', 'Close': 'y'})

# Criando e treinando o modelo
model = Prophet()
model.fit(data)

# Criando um DataFrame com datas futuras para prever
future = model.make_future_dataframe(periods=365)  # Estendendo o período em 365 dias

# Fazendo previsões
forecast = model.predict(future)

# Visualizando as previsões
fig = model.plot(forecast)
plt.show()